In [1]:
import pandas as pd
import geopandas as gp
import numpy as np
from statistics import mean, median
import matplotlib.pyplot as plt
import os
from shapely.geometry import Polygon, MultiPolygon, shape, Point

## VEST documentation

# Wisconsin
---------
Election results and precinct shapefile from Wisconsin State Legislature Open Data Portal (https://data-ltsb.opendata.arcgis.com/)

The following counties were revised with the voting district shapefiles from the U.S. Census Bureau's 2020 Redistricting Data Program Phase 2 release: Buffalo, Clark, Dunn, Grant, La Crosse, Marquette, Pepin, Trempealeau.

G18USSRVUK - Leah Vukmir (Republican Party)
G18USSDBAL - Tammy Baldwin (Democratic Party)
G18USSOWRI - Write-in Votes

G18GOVRWAL - Scott Walker (Republican Party)
G18GOVDEVE - Tony Evers (Democratic Party)
G18GOVLAND - Phillip Anderson (Libertarian Party)
G18GOVGWHI - Michael J. White (Wisconsin Green Party)
G18GOVITUR - Maggie Turnbull (Independent)
G18GOVIENZ - Arnie Enz (Independent)
G18GOVOWRI - Write-in Votes

G18SOSRSCH - Jay Schroeder (Republican Party)
G18SOSDLAF - Doug La Follette (Democratic Party)
G18SOSOWRI - Write-in Votes

G18TRERHAR - Travis Hartwig (Republican Party)
G18TREDGOD - Sarah Godlewski (Democratic Party)
G18TRECZUE - Andrew Zuelke (Constitution Party)
G18TREOWRI - Write-in Votes

G18ATGRSCH - Brad Schimel (Republican Party)
G18ATGDKAU - Josh Kaul (Democratic Party)
G18ATGCLAR - Terry Larson (Constitution Party)
G18ATGOWRI - Write-in Votes


### From personal exchange

"In the listed counties the revisions were minimal. I did not replace all the wards. The revisions mostly involved a few municipal boundaries that had changed between July and November 2018. In La Crosse and Trempealeau there were also a few associated ward adjustments in the adjoining townships. Beyond that I only revised some of the nearby waterlines that were then misaligned between the VTD file and the LTSB file."

## Links for data

Wisconsin wards shapefile link:
https://geodata.wisc.edu/catalog/B41117FE-B7E9-423A-91C8-E0DF28ACA065

Census partnership files for Wisconsin: https://www.census.gov/geo/partnerships/pvs/partnership19v2/st55_wi.html
    

## Load datasets

In [2]:
#Read in the final shapefile from VEST

final = gp.read_file('./final/wi_2018/wi_2018.shp')
#Read in the original wards file
raw_wards = gp.read_file('./raw-data/WI_20122020_Election_Data_Wards_2018/20122020_Election_Data_with_2018_Wards.shp')

#Read in the partneship files for each specified county in the VEST documentation
marquette = gp.read_file('./raw-data/add_counties/marquette/PVS_19_v2_vtd_55077.shp')
trempealeau = gp.read_file('./raw-data/add_counties/trempealeau/PVS_19_v2_vtd_55121.shp')
pepin = gp.read_file('./raw-data/add_counties/pepin/PVS_19_v2_vtd_55091.shp')
buffalo = gp.read_file('./raw-data/add_counties/buffalo/PVS_19_v2_vtd_55011.shp')
clark = gp.read_file('./raw-data/add_counties/clark/PVS_19_v2_vtd_55019.shp')
dunn = gp.read_file('./raw-data/add_counties/dunn/PVS_19_v2_vtd_55033.shp')
grant = gp.read_file('./raw-data/add_counties/grant/PVS_19_v2_vtd_55043.shp')
lacrosse = gp.read_file('./raw-data/add_counties/lacrosse/PVS_19_v2_vtd_55063.shp')

#Set options for pandas displays
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Examine the wards and VEST files

In [3]:
raw_wards.head()

,OBJECTID_1,CNTY_FIPS,CNTY_NAME,COUSUBFP,MCD_FIPS,MCD_NAME,CTV,WARD_FIPS,WARDID,SUPERID,SUPER_FIPS,ALDERID,ALDER_FIPS,CONTACT,DATE_SUB,SCHOOLID,NOTES,LABEL,PERSONS,PERSONS18,WHITE,BLACK,HISPANIC,ASIAN,AMINDIAN,PISLAND,OTHER,OTHERMLT,WHITE18,BLACK18,HISPANIC18,ASIAN18,AMINDIAN18,PISLAND18,OTHER18,OTHERMLT18,GOVTOT18,GOVREP18,GOVDEM18,GOVLIB18,GOVIND18,GOVIND218,GOVIND318,GOVREP218,GOVDEM218,GOVCON18,GOVDEM318,GOVIND418,GOVIND518,GOVDEM418,GOVDEM518,GOVSCT18,SOSTOT18,SOSREP18,SOSDEM18,SOSGRN18,SOSSCT18,TRSTOT18,TRSREP18,TRSDEM18,TRSCON18,TRSSCT18,USHTOT18,USHREP18,USHDEM18,USHIND18,USHIND218,USHDEM218,USHSCT18,USSTOT18,USSREP18,USSDEM18,USSIND18,USSSCT18,USSIND218,WAGTOT18,WAGREP18,WAGDEM18,WAGCON18,WAGSCT18,WSATOT18,WSAREP18,WSADEM18,WSAIND18,WSALIB18,WSASCT18,WSACON18,WSAREP218,WSSTOT18,WSSREP18,WSSDEM18,WSSSCT18,WSSIND18,CDATOT16,CDADEM16,CDADEM216,CDAREP16,CDAIND16,CDASCT16,PRETOT16,PREDEM16,PREREP16,PREGRN16,PRELIB16,PRECON16,PREIND16,PREIND216,PREIND316,PREIND416,PREIND516,PREIND616,PREIND716,PREIND816,PREIND916,PREIND1016,PREIND1116,PRESCT16,USHTOT16,USHDEM16,USHDEM216,USHREP16,USHGRN16,USHLIB16,USHIND16,USHSCT16,USSTOT16,USSDEM16,USSREP16,USSREP216,USSLIB16,USSSCT16,WSATOT16,WSADEM16,WSAREP16,WSALIB16,WSAIND16,WSASCT16,WSSTOT16,WSSDEM16,WSSREP16,WSSIND16,WSSSCT16,GOVTOT14,GOVDEM14,GOVREP14,GOVIND14,GOVIND214,GOVIND314,GOVREP214,GOVCON14,GOVIND414,GOVIND514,GOVREP314,GOVSCT14,SOSTOT14,SOSDEM14,SOSREP14,SOSCON14,SOSIND14,SOSSCT14,TRSTOT14,TRSDEM14,TRSREP14,TRSCON14,TRSIND14,TRSIND214,TRSSCT14,USHTOT14,USHDEM14,USHREP14,USHIND14,USHREP214,USHIND214,USHSCT14,WAGTOT14,WAGDEM14,WAGREP14,WAGIND14,WAGSCT14,WSATOT14,WSADEM14,WSAREP14,WSAIND14,WSAREP214,WSASCT14,WSSTOT14,WSSDEM14,WSSREP14,WSSIND14,WSSSCT14,CDATOT12,CDADEM12,CDADEM212,CDAREP12,CDAIND12,CDASCT12,GOVTOT12,GOVDEM12,GOVREP12,GOVIND12,GOVSCT12,PRETOT12,PREDEM12,PREREP12,PRECON12,PREIND12,PREIND212,PREIND312,PREIND412,PREIND512,PREIND612,PRESCT12,USHTOT12,USHDEM12,USHREP12,USHIND12,USHSCT12,USSTOT12,USSDEM12,USSREP12,USSCON12,USSIND12,USSIND212,USSIND312,USSSCT12,WSATOT12,WSADEM12,WSADEM212,WSAREP12,WSAREP212,WSAIND12,WSAIND212,WSASCT12,WSSTOT12,WSSDEM12,WSSREP12,WSSREP212,WSSCON12,WSSIND12,WSSSCT12,WSSAME12,ASM,SEN,CON,geometry
0,1,55001,ADAMS,00275,5500100275,Adams,C,55001002750001,0001,11,5500111,01,550010027501,afaust@ncwrpc.org,7/12/2018 9:06:33 AM,None,None,Adams - C 0001,661,484,620,17,15,1,5,0,0,3,463,9,8,1,3,0,0,0,228,120,99,3,2,2,1,0,0,0,0,0,0,0,0,1,218,110,108,0,0,217,106,102,9,0,220,100,120,0,0,0,0,223,103,119,0,1,0,222,114,102,6,0,220,125,95,0,0,0,0,0,0,0,0,0,0,183,0,0,0,180,3,258,110,127,3,6,3,1,1,0,2,0,0,0,0,0,0,0,5,164,162,0,0,0,0,0,2,245,111,116,0,18,0,217,0,126,0,91,0,227,121,106,0,0,193,101,85,3,4,0,0,0,0,0,0,0,180,106,65,4,5,0,174,91,70,3,6,4,0,179,109,70,0,0,0,0,182,94,81,7,0,181,94,87,0,0,0,0,0,0,0,0,193,0,0,0,191,2,216,104,95,15,2,272,168,101,1,1,0,1,0,0,0,0,237,150,86,0,1,262,148,99,0,11,3,0,1,232,140,0,92,0,0,0,0,230,137,92,0,0,0,1,0,41,14,3,"POLYGON ((533814.917 388202.113, 533814.915 38..."
1,2,55001,ADAMS,00275,5500100275,Adams,C,55001002750002,0002,12,5500112,02,550010027502,afaust@ncwrpc.org,7/12/2018 9:06:33 AM,None,None,Adams - C 0002,652,463,599,6,23,10,12,0,2,0,435,4,7,7,8,0,2,0,219,118,97,1,1,2,0,0,0,0,0,0,0,0,0,0,216,109,107,0,0,212,104,99,9,0,215,98,117,0,0,0,0,218,101,117,0,0,0,214,111,99,4,0,214,122,92,0,0,0,0,0,0,0,0,0,0,178,0,0,0,177,1,241,107,124,1,4,1,0,0,0,1,0,0,0,0,0,0,0,3,160,159,0,0,0,0,0,1,239,109,113,0,17,0,212,0,123,0,89,0,221,118,103,0,0,185,98,84,1,2,0,0,0,0,0,0,0,171,103,63,2,3,0,166,88,68,2,5,3,0,175,106,69,0,0,0,0,177,91,79,7,0,176,92,84,0,0,0,0,0,0,0,0,188,0,0,0,188,0,169,89,80,0,0,262,164,98,0,0,0,0,0,0,0,0,230,147,83,0,0,253,145,96,0,10,2,0,0,226,137,0,89,0,0,0,0,224,135,89,0,0,0,0,0,41,14,3,"POLYGON ((535291.528 386599.081, 534978.372 38..."
2,3,55001,ADAMS,00275,5500100275,Adams,C,55001002750003,0003,12,5500112,03,550010027503,afaust@ncwrpc.org,7/12/2018 9:06:33 AM,None,None,Adams - C 0003,288,241,278,6,3,0,1,0,

In [4]:
final.head()

,OBJECTID_1,CNTY_FIPS,CNTY_NAME,COUSUBFP,MCD_FIPS,MCD_NAME,WARDID,LABEL,ASM,SEN,CON,PERSONS,PERSONS18,WHITE,BLACK,HISPANIC,ASIAN,AMINDIAN,PISLAND,OTHER,OTHERMLT,WHITE18,BLACK18,HISPANIC18,ASIAN18,AMINDIAN18,PISLAND18,OTHER18,OTHERMLT18,G18USSRVUK,G18USSDBAL,G18USSOWRI,G18GOVRWAL,G18GOVDEVE,G18GOVLAND,G18GOVGWHI,G18GOVITUR,G18GOVIENZ,G18GOVOWRI,G18SOSRSCH,G18SOSDLAF,G18SOSOWRI,G18TRERHAR,G18TREDGOD,G18TRECZUE,G18TREOWRI,G18ATGRSCH,G18ATGDKAU,G18ATGCLAR,G18ATGOWRI,geometry
0,1,55001,ADAMS,00275,5500100275,Adams,0001,Adams - C 0001,41,14,3,661,484,620,17,15,1,5,0,0,3,463,9,8,1,3,0,0,0,103,119,1,120,99,3,2,2,1,1,110,108,0,106,102,9,0,114,102,6,0,"POLYGON ((-89.82778 43.96682, -89.82778 43.966..."
1,2,55001,ADAMS,00275,5500100275,Adams,0002,Adams - C 0002,41,14,3,652,463,599,6,23,10,12,0,2,0,435,4,7,7,8,0,2,0,101,117,0,118,97,1,1,2,0,0,109,107,0,104,99,9,0,111,99,4,0,"POLYGON ((-89.80942 43.95236, -89.81333 43.952..."
2,3,55001,ADAMS,00275,5500100275,Adams,0003,Adams - C 0003,41,14,3,288,241,278,6,3,0,1,0,0,0,234,4,2,0,1,0,0,0,45,52,1,52,44,1,1,1,1,1,48,47,0,46,44,4,0,50,44,3,0,"POLYGON ((-89.79772 43.96676, -89.79771 43.965..."
3,4,55001,ADAMS,00275,5500100275,Adams,0004,Adams - C 0004,41,14,3,366,306,350,2,5,2,5,0,1,1,298,1,2,2,2,0,0,1,56,65,0,66,54,1,0,0,0,0,60,59,0,58,56,4,0,62,56,2,0,"POLYGON ((-89.81786 43.96681, -89.81777 43.964..."
4,5,55001,ADAMS,00300,5500100300,ADAMS,0001,Adams - T 0001,41,14,3,902,736,847,2,38,2,11,0,1,1,704,2,21,2,5,0,1,1,212,190,0,239,156,6,4,2,0,0,223,164,0,218,155,14,0,231,155,8,0,"POLYGON ((-89.71669 43.89442, -89.71699 43.894..."


In [5]:
print(list(final.columns))
print('\n')
print(list(raw_wards.columns))

['OBJECTID_1', 'CNTY_FIPS', 'CNTY_NAME', 'COUSUBFP', 'MCD_FIPS', 'MCD_NAME', 'WARDID', 'LABEL', 'ASM', 'SEN', 'CON', 'PERSONS', 'PERSONS18', 'WHITE', 'BLACK', 'HISPANIC', 'ASIAN', 'AMINDIAN', 'PISLAND', 'OTHER', 'OTHERMLT', 'WHITE18', 'BLACK18', 'HISPANIC18', 'ASIAN18', 'AMINDIAN18', 'PISLAND18', 'OTHER18', 'OTHERMLT18', 'G18USSRVUK', 'G18USSDBAL', 'G18USSOWRI', 'G18GOVRWAL', 'G18GOVDEVE', 'G18GOVLAND', 'G18GOVGWHI', 'G18GOVITUR', 'G18GOVIENZ', 'G18GOVOWRI', 'G18SOSRSCH', 'G18SOSDLAF', 'G18SOSOWRI', 'G18TRERHAR', 'G18TREDGOD', 'G18TRECZUE', 'G18TREOWRI', 'G18ATGRSCH', 'G18ATGDKAU', 'G18ATGCLAR', 'G18ATGOWRI', 'geometry']


['OBJECTID_1', 'CNTY_FIPS', 'CNTY_NAME', 'COUSUBFP', 'MCD_FIPS', 'MCD_NAME', 'CTV', 'WARD_FIPS', 'WARDID', 'SUPERID', 'SUPER_FIPS', 'ALDERID', 'ALDER_FIPS', 'CONTACT', 'DATE_SUB', 'SCHOOLID', 'NOTES', 'LABEL', 'PERSONS', 'PERSONS18', 'WHITE', 'BLACK', 'HISPANIC', 'ASIAN', 'AMINDIAN', 'PISLAND', 'OTHER', 'OTHERMLT', 'WHITE18', 'BLACK18', 'HISPANIC18', 'ASIAN18', 'AMIN

## Rename the wards dataset fields to match those in the VEST file

In [6]:
col_dict = {'OBJECTID':'OBJECTID_1','WARD_ID':'WARDID','USSREP18':'G18USSRVUK', 
            'USSDEM18':'G18USSDBAL','GOVREP18':'G18GOVRWAL','GOVDEM18': 'G18GOVDEVE',
            'GOVLIB18':'G18GOVLAND','GOVIND18':'G18GOVGWHI','GOVIND218':'G18GOVITUR',
            'GOVIND318':'G18GOVIENZ','SOSREP18':'G18SOSRSCH','SOSDEM18':'G18SOSDLAF',
            'TRSREP18':'G18TRERHAR','TRSDEM18':'G18TREDGOD','TRSCON18':'G18TRECZUE',
            'WAGREP18':'G18ATGRSCH','WAGDEM18':'G18ATGDKAU','WAGCON18':'G18ATGCLAR'}

wards = raw_wards.rename(col_dict, axis=1)


## Calculate the 'write-in' vote fields

In [7]:
wards['G18USSOWRI'] = wards['USSSCT18']+wards['USSIND18']+wards['USSIND218']
wards['G18GOVOWRI'] = wards['GOVREP218'] + wards['GOVDEM218']+wards['GOVCON18']+wards['GOVDEM318']+wards['GOVIND418']+wards['GOVIND518']+wards['GOVDEM418']+wards['GOVDEM518']+wards['GOVSCT18']
wards['G18SOSOWRI'] = wards['SOSGRN18']+wards['SOSSCT18']
wards['G18TREOWRI']=wards['TRSSCT18']
wards['G18ATGOWRI']=wards['WAGSCT18']

## Subset the wards to match the columns in the VEST file

In [8]:
wards = wards[['OBJECTID_1', 'CNTY_FIPS', 'CNTY_NAME', 'COUSUBFP',
                    'MCD_FIPS', 'MCD_NAME', 'WARDID','LABEL',
                    'ASM','SEN','CON','PERSONS','PERSONS18','WHITE',
                    'BLACK', 'HISPANIC', 'ASIAN', 'AMINDIAN', 'PISLAND',
                    'OTHER','OTHERMLT', 'WHITE18','BLACK18','HISPANIC18',
                    'ASIAN18','AMINDIAN18','PISLAND18','OTHER18',
                    'OTHERMLT18','G18USSRVUK','G18USSDBAL','G18USSOWRI',
                    'G18GOVRWAL','G18GOVDEVE','G18GOVLAND','G18GOVGWHI',
                    'G18GOVITUR','G18GOVIENZ','G18GOVOWRI','G18SOSRSCH',
                    'G18SOSDLAF','G18SOSOWRI','G18TRERHAR','G18TREDGOD',
                    'G18TRECZUE','G18TREOWRI','G18ATGRSCH','G18ATGDKAU',
                    'G18ATGCLAR','G18ATGOWRI','geometry']]

## Confirm the columns are the same between the two files

In [9]:
col_check_dict = dict(zip(list(wards),list(final)))
matching = 0
not_matching = 0
for k,v in col_check_dict.items():
    if k==v:
        matching = matching+1
    else:
        not_matching = not_matching+1
print('There are ', matching, ' columns that have the same name and location in the final and wards files. \nThere are ', not_matching, ' columns that do NOT have the same name and/or location in the final and wards files.')

There are  51  columns that have the same name and location in the final and wards files. 
There are  0  columns that do NOT have the same name and/or location in the final and wards files.


## Check the row and column totals before validating

In [10]:
print('Wards file shape: ', wards.shape)
print('VEST file shape: ', final.shape)
print('Do these datasets have the same number of rows and columns? ', str(final.shape==wards.shape))

Wards file shape:  (6975, 51)
VEST file shape:  (6975, 51)
Do these datasets have the same number of rows and columns?  True


# VALIDATION

## Check the geographies between the files

### Add unique ID column to sort the wards on

In [11]:
wards['JOIN'] = wards.apply(lambda x: ' '.join([x['CNTY_FIPS'],x['COUSUBFP'],x['MCD_FIPS'],x['LABEL']]),axis=1)
final['JOIN'] = final.apply(lambda x: ' '.join([x['CNTY_FIPS'],x['COUSUBFP'],x['MCD_FIPS'],x['LABEL']]),axis=1)
wards['REPLACE'] = wards.apply(lambda x: ' '.join([x['CNTY_FIPS'],x['LABEL'].upper()]),axis=1)
final['REPLACE'] = final.apply(lambda x: ' '.join([x['CNTY_FIPS'],x['LABEL'].upper()]),axis=1)

### Validate that the unique IDs are the same between the two files

In [12]:
def check_ids(df1, df2, verify_field):
    unique_final = []
    for i in df1[verify_field]:
        unique_final.append(i)
    unique_prec = []
    for i in df2[verify_field]:
        unique_prec.append(i)
    differing = []
    for i in unique_final:
        if i not in unique_prec:
            print(i, ' not in df1 file')
            differing.append(i)
    for i in unique_prec:
        if i not in unique_final:
            print(i, ' not in df2 file')
            differing.append(i)

    print('Numer of unique values in ',verify_field, ' filed in the wards file: ', str(df2[verify_field].nunique()))

    print('There are ', len(differing), ' wards that are different between the two files.')
    if len(differing)!=0:
          print('The differing wards are: ', ''.join(differing))
    matching = final.shape[0]==wards.shape[0]
    print('Are there the same number of cases in df1 and df2 (T/F)? ', str(matching))

### Run the function on the VEST and wards files

In [13]:
check_ids(final,wards,'JOIN')

Numer of unique values in  JOIN  filed in the wards file:  6975
There are  0  wards that are different between the two files.
Are there the same number of cases in df1 and df2 (T/F)?  True


## Verify that the geometries are the same between the VEST and wards file

### Write a function to check geometries similarity

In [14]:
def validate_geom(gdf1,gdf2,sort_field,proj_epsg,dec=0):
    gdf1["geometry"]=gdf1.buffer(0) #buffer by 0 to remove any non polygons
    gdf2["geometry"]=gdf2.buffer(0)
    proj = gdf1.crs #project to same projection
    gdf2 = gdf2.to_crs(proj)
    gdf1 = gdf1.to_crs(proj)
    gdf1 = gdf1.sort_values(by=[sort_field]) #sort values by the unique field
    gdf2 = gdf2.sort_values(by=[sort_field])
    gdf2.reset_index(drop=True,inplace=True) #drop the index
    gdf1.reset_index(drop=True,inplace=True)
    ###Add in print link difference between the two files
    ## Look at GA example
    #see the difference between the precinct and final file
    ngdf1 = gdf1.to_crs(epsg=proj_epsg)
    ngdf2 = gdf2.to_crs(epsg=proj_epsg)
    ngdf1 = ngdf1.buffer(0)
    ngdf2 = ngdf2.buffer(0)
    file = ngdf2.difference(ngdf1)
    print('Difference between gdf1 and gdf2 as a percent of gdf1 total area ', str((sum(file.area)/sum(ngdf1.area))*100))
    print('Difference between gdf1 and gdf2 as a percent of gdf2 total area ', str((sum(file.area)/sum(ngdf2.area))*100))
    #To 6 decimal places, the two files are equal 
    t_f_geom = list(gdf1.geom_almost_equals(gdf2,decimal=dec))
    gdf1['GEOM_EQUALS'] = t_f_geom
    gdf2['GEOM_EQUALS'] = t_f_geom
    unique_geom = []
    for i in t_f_geom:
        if i not in unique_geom:
            unique_geom.append(i)
    for i in unique_geom:
        percent = str((t_f_geom.count(i)/len(t_f_geom))*100) 
        print(str(t_f_geom.count(i)) + ' wards are ' + str(i) + ' which is ' + percent + ' percent of all wards.')
    return gdf1, gdf2

In [15]:
geom_validate1 = validate_geom(final,wards,'JOIN',3070)

Difference between gdf1 and gdf2 as a percent of gdf1 total area  0.007172107026069504
Difference between gdf1 and gdf2 as a percent of gdf2 total area  0.0071757668003647485
6749 wards are True which is 96.75985663082437 percent of all wards.
226 wards are False which is 3.240143369175627 percent of all wards.


## Modify the wards shapefile with the Census VTDs

### Join all VTDs retrieved together

In [16]:
census_vtds = gp.GeoDataFrame(pd.concat([marquette,trempealeau,pepin,buffalo,clark,dunn,grant,lacrosse]),crs = marquette.crs)

### Write function to convert all 3D geometries to 2D geometries

In [17]:
def convert_3D_2D(gdf):
    '''
    Takes a GeoSeries of 3D Multi/Polygons (has_z) and returns a list of 2D Multi/Polygons
    '''
    geometry = gdf['geometry']
    new_geo = []
    for p in geometry:
        if p.has_z:
            if p.geom_type == 'Polygon':
                lines = [xy[:2] for xy in list(p.exterior.coords)]
                new_p = Polygon(lines)
                new_geo.append(new_p)
            elif p.geom_type == 'MultiPolygon':
                new_multi_p = []
                for ap in p:
                    lines = [xy[:2] for xy in list(ap.exterior.coords)]
                    new_p = Polygon(lines)
                    new_multi_p.append(new_p)
                new_geo.append(MultiPolygon(new_multi_p))
        else:
            new_geo.append(p)
    gdf['geometry'] = new_geo
    return gdf

### Run the 3D to 2D function for the shapefiles

In [18]:
census_vtds = convert_3D_2D(census_vtds) # new geodf with 2D geometry series
final = geom_validate1[0]
wards = geom_validate1[1]
final = convert_3D_2D(final)
wards = convert_3D_2D(wards)

### Create field to use for replacing data from VTD file to wards file

In [19]:
census_vtds['CNTY_FIPS'] = census_vtds.apply(lambda x: ''.join([x['STATEFP'],x['COUNTYFP']]),axis=1)
census_vtds['REPLACE'] = census_vtds.apply(lambda x: ' '.join([x['CNTY_FIPS'],x['NAME'].upper()]),axis=1)

### Subset the retrieved gdfs from the validate_geom function to see what was *NOT* equal

In [20]:
final = geom_validate1[0]
wards = geom_validate1[1]
final_to_edit = final[final['GEOM_EQUALS'].isin([False])]
wards_to_edit = wards[wards['GEOM_EQUALS'].isin([False])]

### Identify the wards that need to be supplemented and clean the data and append them to the wards gdf

In [21]:
wards_list = list(wards_to_edit['REPLACE'])

census_vtds_replace = census_vtds[census_vtds['REPLACE'].isin(wards_list)]

wards_fixed = pd.merge(census_vtds_replace,wards_to_edit, on = 'REPLACE',how='inner')
wards_fixed = wards_fixed[['OBJECTID_1','CNTY_FIPS_y','CNTY_NAME','COUSUBFP','MCD_FIPS','MCD_NAME','WARDID','LABEL','ASM',
                          'SEN','CON','PERSONS','PERSONS18','WHITE','BLACK','HISPANIC','ASIAN','AMINDIAN','PISLAND','OTHER',
                          'OTHERMLT','WHITE18','BLACK18','HISPANIC18','ASIAN18','AMINDIAN18','PISLAND18','OTHER18','OTHERMLT18',
                          'G18USSRVUK','G18USSDBAL','G18USSOWRI','G18GOVRWAL','G18GOVDEVE','G18GOVLAND','G18GOVGWHI','G18GOVITUR',
                          'G18GOVIENZ','G18GOVOWRI','G18SOSRSCH','G18SOSDLAF','G18SOSOWRI','G18TRERHAR','G18TREDGOD','G18TRECZUE',
                          'G18TREOWRI','G18ATGRSCH','G18ATGDKAU','G18ATGCLAR','G18ATGOWRI','JOIN','REPLACE','GEOM_EQUALS','geometry_x']]
wards_fixed.rename(columns={'geometry_x':'geometry','CNTY_FIPS_y':'CNTY_FIPS'},inplace=True)
wards_fixed_ids = list(wards_fixed['JOIN'])

wards_removed = wards[~wards['JOIN'].isin(wards_fixed_ids)]
wards_edited = gp.GeoDataFrame(pd.concat([wards_removed,wards_fixed]),crs= wards_removed.crs)

### Re-run the validate geom function with the edited ward boundaries

In [22]:
geom_validate2 = validate_geom(final,wards_edited,'JOIN',3070)
final = geom_validate2[0]
wards = geom_validate2[1]

Difference between gdf1 and gdf2 as a percent of gdf1 total area  0.014639195784988462
Difference between gdf1 and gdf2 as a percent of gdf2 total area  0.014640559597804522
6859 wards are True which is 98.33691756272401 percent of all wards.
116 wards are False which is 1.6630824372759856 percent of all wards.


## Check the column totals

## Join the final VEST file with the ward shapefile with election results

In [23]:
validate = wards.merge(final, on='JOIN',how='outer',indicator=True) #merge on the 'Final Join column'

print('Number of wards in the joined precincts dataset: ', str(wards.shape[0]))
print('Number of wards in the final VEST dataset: ', str(final.shape[0]))
print('Number of wards in the wards and final VEST file : ',str(validate.shape[0]))
print('Wards with election results that were successfully joined with the final file as a percentage of the wards and election results geodataframe: ', str((validate.shape[0]/wards.shape[0])*100))
print('Wards with election results that were successfully joined with the final file as a percentage of the wards in the final file: ', str((validate.shape[0]/final.shape[0])*100))

Number of wards in the joined precincts dataset:  6975
Number of wards in the final VEST dataset:  6975
Number of wards in the wards and final VEST file :  6975
Wards with election results that were successfully joined with the final file as a percentage of the wards and election results geodataframe:  100.0
Wards with election results that were successfully joined with the final file as a percentage of the wards in the final file:  100.0


### Write a function to check the column totals

In [24]:
def sum_checker (df, column_List):
    """Checks the sums of different columns and prints out the maximum difference by column. 
    Assumes that the dataframe is the merged dataframe between RDH and the partner, with the same named election result
    columns, except RDH is _x and Partner is _y. 
    """
    list_max = 0
    for i in column_List:
        left_sum = df[i+"_x"].sum()
        right_sum = df[i+"_y"].sum()
        if (left_sum==right_sum):
            print("Column: ", i, " have the same sums")
            
        else:
            diff = abs(left_sum-right_sum)
            percent = abs(left_sum-right_sum)/left_sum
            print("Column: ", i)
            print("RDH sum is ", str(left_sum))
            print("Partner sum is ", str(right_sum))
            print("Their (absolute) difference is ",str(diff))
            print("As a percentage of DF1 this is ", str(round(percent*100,5)) + "%")
            if (percent>list_max):
                list_max = percent
            print("")
    print("The max difference (as a percent of the total of one DF column) is: ", str(round(list_max,5)))

### Run the column total function for the joined gdf

In [25]:
column_List = ['PERSONS', 'PERSONS18', 'WHITE', 'BLACK', 'HISPANIC', 'ASIAN', 'AMINDIAN', 'PISLAND', 'OTHER', 'OTHERMLT', 'WHITE18', 'BLACK18', 'HISPANIC18', 'ASIAN18', 'AMINDIAN18', 'PISLAND18', 'OTHER18', 'OTHERMLT18', 'G18USSRVUK', 'G18USSDBAL', 'G18USSOWRI', 'G18GOVRWAL', 'G18GOVDEVE', 'G18GOVLAND', 'G18GOVGWHI', 'G18GOVITUR', 'G18GOVIENZ', 'G18GOVOWRI', 'G18SOSRSCH', 'G18SOSDLAF', 'G18SOSOWRI', 'G18TRERHAR', 'G18TREDGOD', 'G18TRECZUE', 'G18TREOWRI', 'G18ATGRSCH', 'G18ATGDKAU', 'G18ATGCLAR', 'G18ATGOWRI']
sum_checker(validate, column_List)

Column:  PERSONS  have the same sums
Column:  PERSONS18  have the same sums
Column:  WHITE  have the same sums
Column:  BLACK  have the same sums
Column:  HISPANIC  have the same sums
Column:  ASIAN  have the same sums
Column:  AMINDIAN  have the same sums
Column:  PISLAND  have the same sums
Column:  OTHER  have the same sums
Column:  OTHERMLT  have the same sums
Column:  WHITE18  have the same sums
Column:  BLACK18  have the same sums
Column:  HISPANIC18  have the same sums
Column:  ASIAN18  have the same sums
Column:  AMINDIAN18  have the same sums
Column:  PISLAND18  have the same sums
Column:  OTHER18  have the same sums
Column:  OTHERMLT18  have the same sums
Column:  G18USSRVUK  have the same sums
Column:  G18USSDBAL  have the same sums
Column:  G18USSOWRI  have the same sums
Column:  G18GOVRWAL  have the same sums
Column:  G18GOVDEVE  have the same sums
Column:  G18GOVLAND  have the same sums
Column:  G18GOVGWHI  have the same sums
Column:  G18GOVITUR  have the same sums
Column

### Write a function for ward-by-ward comparision of the election and demographic data

In [26]:
def validater_row_vect(df, join_column, column_List):
    matching_rows = 0
    different_rows = 0
        
    join_cols = (df[join_column]).astype('str').to_numpy()
    
    # it's because before, we were checking that a row was entirely consistent
    # here, we are double checking a lot
    # so, let's just keep one track of the rows that are messed up
    
    wrong_idxs = np.zeros(len(df))
    for i in column_List:
        left_data = df[i + "_x"].to_numpy()
        right_data = df[i + "_y"].to_numpy()
        
        local_idxs = np.where(left_data != right_data)
        wrong_idxs[local_idxs] = 1
        #print("Wrong idxs", wrong_idxs)
        
    # we are close, we get the same result, but are double adding lots of rows
    different_rows += np.sum(wrong_idxs)
    matching_rows += len(df) - different_rows
    
    diff_list = join_cols[np.where(wrong_idxs == 1)]
            
    print("There are ", len(df.index)," total rows")
    print(f"{int(different_rows)} of these rows have election or demographic result differences")
    print(f"{int(matching_rows)} of these rows are the same")
    print(diff_list)
    print("")

### Run the ward-by-ward function for the joined gdf

In [27]:
validater_row_vect(validate, 'JOIN', column_List)

There are  6975  total rows
0 of these rows have election or demographic result differences
6975 of these rows are the same
[]

